In [ ]:
#Script Based on: https://github.com/mousepixels/sanbomics_scripts/blob/main/scVI_tools_introduction.ipynb

#dependencies
!pip install scvi-tools
!pip install scikit-misc
!pip install scanpy
!pip install leidenalg

In [ ]:
import scvi
import scanpy as sc
from scipy import io
import anndata
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pyplot import rc_context
sc.set_figure_params(dpi=100)

In [ ]:
X = io.mmread("Data_counts.mtx")

# create anndata object
adata = anndata.AnnData(X=X.transpose().tocsr())

# load cell metadata:
cell_meta = pd.read_csv("Data_metadata.csv")

# load gene names:
with open("Data_gene_names.csv", 'r') as f:
    gene_names = f.read().splitlines()

# set anndata observations and index obs by barcodes, var by gene names
adata.obs = cell_meta
adata.obs.index = adata.obs['barcode']
adata.var.index = gene_names

# save dataset as anndata format
adata.write('Data_Python.h5ad')

#If you need to do the QC
sc.pp.filter_cells(adata, min_genes = 200)
sc.pp.filter_genes(adata, min_cells = 3)
adata.var['mt'] = adata.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata, qc_vars = ['mt'], percent_top = None, log1p = False, inplace = True)
adata = adata[adata.obs.pct_counts_mt < 15]

In [ ]:
adata.layers['counts'] = adata.X.copy() #IMPORTANT, this is used by scVI. This will not change
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
adata.raw = adata

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset = True, layer = 'counts',
                           flavor = "seurat_v3", batch_key="Age") #no batch_key if one sample

scvi.model.SCVI.setup_anndata(adata, layer = "counts",
                             categorical_covariate_keys=["Age"],
                             continuous_covariate_keys=['percent.mt', 'nCount_RNA'])
model = scvi.model.SCVI(adata)
model.train() #may take a while without GPU

In [ ]:
latent = model.get_latent_representation() #this is what you will use to cluster now instead of PCs like normal
latent.shape
adata.obsm['X_scVI'] = latent
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)
sc.pp.neighbors(adata, use_rep = 'X_scVI')
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution = 0.5)

In [ ]:
with plt.rc_context({'figure.figsize': (4, 4)}):
    sc.pl.umap(adata, color=['leiden', 'Age'], frameon=False)
    # Salvar a figura
    plt.savefig('umap_plot.png')

In [ ]:
#find markers of each cluster
df = model.differential_expression(groupby = 'leiden')
df
markers = {}
for c in adata.obs.leiden.cat.categories:
    cell_df = df.loc[df.group1 == c]
    markers[c] = cell_df.index.tolist()[:2]
sc.pl.dotplot(adata, markers, groupby = 'leiden', swap_axes = True,
             use_raw = True, standard_scale = 'var', dendrogram = True, save = "dotplot.png")